# Fatbox for Analogue Modelling - Elevation vs Strain - tuto 5/6

This tutorial follows Tutorials B2 and B4 on how to extract a 2-D fault network from an analogue model simulating orthogonal continental rifting.

Here we will load the clean graphs created in the mentioned tutorials and we are going to plot them to compare.

The analog models are based on the study of Molnar et al. (2017)


## Load packages
To run the toolbox, we will need a couple of packages including the toolbox itself. So let's install it:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pwd

Mounted at /content/drive
/content


In [ ]:
!pip3 install earthpy
!pip install cv-algorithms
!pip install vtk

  Using cached cv_algorithms-1.1.1.tar.gz (22 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for cv-algorithms: filename=cv_algorithms-1.1.1-cp311-cp311-manylinux_2_35_x86_64.whl size=40944 sha256=f6546bf5bf05346ec44fb1543bc1a961852c0c0bd70544b74c2e959ac2886109
  Stored in directory: /root/.cache/pip/wheels/6d/fc/8d/7dc6a93ee79ff25fd3491eac76e255cca24893bb3d5f3cc150
Successfully built cv-algorithms


Now we can load the python packages that we need:

In [ ]:
import numpy as np
import networkx as nx
import pickle

import matplotlib.pyplot as plt

from ipywidgets import Layout, interactive, widgets
from tqdm import tqdm

import earthpy.spatial as es

from pathlib import Path #The gestion of Path with pathlib allows for universal use.
import os
path_folder=Path('/content/drive/MyDrive/Fatbox')
path_folder=path_folder/'modules'
#print(path_folder) #make sure path_folder = '/Fatbox_fault_analysis_toolbox/modules'
os.chdir(path_folder) # make modules as working directory
#type pwd in console and make sure it is '/Fatbox_fault_analysis_toolbox/modules'

import preprocessing
import metrics
import plots
import utils
import structural_analysis
import edits

## Load data

In [ ]:
#unpickle our data here
save_path = Path('/content/drive/MyDrive/Fatbox/tutorials/analog')
loc=(save_path/'array')

with open((loc/"dem_graphs.pkl"), "rb") as fp:   # Unpickling fault network extracted from elevation data
    Gs = pickle.load(fp)

with open((loc/"is_graphs.pkl"), "rb") as fq:   # Unpickling fault network extracted from strain data
    Hs = pickle.load(fq)

with open((loc/"dems.pkl"), "rb") as fr:   # Unpickling fault network extracted from strain data
    dems = pickle.load(fr)

And our DEMs to use as a background for visualization purposes:

In [ ]:
path_input=Path('/content/drive/MyDrive/Fatbox/tutorials/analog/data_analog')

dems = [] #all dem arrays will be stored here

for n in range(1,17): #because there are 16 arrays, dem1.npy through dem16.npy
  data = np.load((path_input/str('dem' + str(n) + '.npy')))
  data = data[:,:-1] #added this line because the canny edge detection algorithm was generating an unreal 'fault' in the right hand side of the arrays
  dems.append(data)


## 1. Comparison plot

In [ ]:
az = 300 #azimuth of the light for the hillshade
alt = 1 #altitute (degrees) of the light for the hillshade

def f(time):
    G = Gs[time] #get fault network from DEM (Tutorial B2)
    H = Hs[time] #get fault network from strain (Tutorial B4)
    dem = dems[time]
    hillshade = es.hillshade(dem, azimuth=az, altitude=alt) #create hillshade for better visualisation

    fig, axs = plt.subplots(1,3, figsize=(12,8)) #set up plot

    plots.plot_faults(G, node_size=1, ax=axs[0], label=True)
    #axs[0].imshow(dem, cmap='gist_earth', vmin=-8, vmax=2, alpha=0.4) #plot raw DEM
    #axs[0].imshow(hillshade, cmap='Greys', alpha=0.2)
    axs[0].set_title('Fault network from DEM')
    axs[0].set_ylim([1000,0])
    axs[0].set_xlim([0,400])
    #axs[0].axis('off')

    axs[1].set_title('DEM')
    axs[1].imshow(dem, cmap='gist_earth', vmin=-8, vmax=2)
    axs[1].imshow(hillshade, cmap='Greys', alpha=0.4)
    axs[1].set_ylim([1000,0])
    axs[1].set_xlim([0,400])
    axs[1].axis('off')

    plots.plot_faults(H, node_size=1, ax=axs[2], label=True)
    #axs[2].imshow(dem, cmap='gist_earth', vmin=-8, vmax=2, alpha=0.4)
    #axs[2].imshow(hillshade, cmap='Greys', alpha=0.2)
    axs[2].set_title('Fault network from strain')
    axs[2].set_ylim([1000,0])
    axs[2].set_xlim([0,400])
    #axs[2].axis('off')

    plt.tight_layout()
    plt.show()

interactive_plot = interactive(f, time=widgets.IntSlider(min=0, max=14, step=1, layout=Layout(width='700px')))
output = interactive_plot.children[-1]
output.layout.width = '800px'
interactive_plot

interactive(children=(IntSlider(value=0, description='time', layout=Layout(width='700px'), max=14), Output(lay…